In [29]:
import os
import json
import pandas as pd
import traceback


In [30]:
from langchain.chat_models import ChatOpenAI # langchain_community needed to be installed, manual installed it here


In [31]:
from dotenv import load_dotenv
load_dotenv()  # take environment varibles from .env


True

In [32]:
Key=os.getenv("OPENAI_API_KEY")


In [ ]:
print(Key)


In [34]:
llm=ChatOpenAI(openai_api_key=Key, mode_name="gpt-3.5-turbo", temperature=0.5)


WARNING! mode_name is not default parameter.
                    mode_name was transferred to model_kwargs.
                    Please confirm that mode_name is what you intended.


In [35]:
llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C3674F6790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C367168D90>, temperature=0.5, model_kwargs={'mode_name': 'gpt-3.5-turbo'}, openai_api_key='sk-tlW4PSjWeCmwgh9FolF0mgxu98HG05VNFASrEg3k6QT3BlbkFJZxvAff81GE642ImVZ7cnU1uOaGSDqC8aHsDKe44q0A', openai_proxy='')

In [36]:
from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


# initial jobs basically ended here


In [37]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [38]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [39]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n')

In [40]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C3674F6790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C367168D90>, temperature=0.5, model_kwargs={'mode_name': 'gpt-3.5-turbo'}, openai_api_key='sk-tlW4PSjWeCmwgh9FolF0mgxu98HG05VNFASrEg3k6QT3BlbkFJZxvAff81GE642ImVZ7cnU1uOaGSDqC8aHsDKe44q0A', openai_proxy=''), output_key='quiz')

In [41]:
quiz_chain


LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C3674F6790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C367168D90>, temperature=0.5, model_kwargs={'mode_name': 'gpt-3.5-turbo'}, openai_api_key='sk-tlW4PSjWeCmwgh9FolF0mgxu98HG05VNFASrEg3k6QT3BlbkFJZxvAff81GE642ImVZ7cnU1uOaGSDqC8aHsDKe44q0A', openai_proxy=''), output_key='quiz')

In [42]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [51]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
            output_variables=["quiz", "review"], verbose=True,) 
file_path=r"C:\Users\simon9k\Documents\2024AIDev\MCQGEN\data.txt"
file_path

'C:\\Users\\simon9k\\Documents\\2024AIDev\\MCQGEN\\data.txt'

In [44]:
with open(file_path, 'r') as file:
    TEXT = file.read()
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [45]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [46]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking


In [52]:

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

TypeError: create() got an unexpected keyword argument 'mode_name'

In [48]:
response


NameError: name 'response' is not defined